In [ ]:
import fasttext
import pandas as pd
import numpy as np
import itertools
import pickle
import os

import fasttext
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

from sklearn.impute import KNNImputer

from imblearn.metrics import specificity_score
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report
from imblearn.combine import SMOTETomek
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier

from boruta import BorutaPy

import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

In [ ]:
df = pd.read_excel('../../data/coorte-rues-2022-2023-V3_com_extracoes.xlsx')
df = df.drop('Unnamed: 0', axis=1)

### Preprocessing - Tempo de Permanência: All data

In [ ]:
df.fillna({
    'FC__anamnese_medica': 'NORMOCARDICO',
    'FR__anamnese_medica': 'EUPNEIA',
    'PA__anamnese_medica': 'NORMOTENSAO',
    'T__anamnese_medica': 'NORMOTERMIA',
    'SpO2__anamnese_medica': 'NORMOXEMIA',
    'SpO2__anamnese_enfermagem': 'NORMOXEMIA',
    'R__anamnese_medica': 'LUCIDO E ORIENTADO',
    'FC__anamnese_enfermagem': 'NORMOCARDICO',
    'FR__anamnese_enfermagem': 'EUPNEIA',
    'PA__anamnese_enfermagem': 'NORMOTENSAO',
    'T__anamnese_enfermagem': 'NORMOTERMIA',
    'R__anamnese_enfermagem': 'LUCIDO E ORIENTADO'
}, inplace=True)

In [ ]:
df = df.sort_values(by=['prontuario', 'data_internacao'])

In [ ]:
df_treino = df[df['data_internacao'].astype(str).str.contains('2023|2022')]
df_teste = df[df['data_internacao'].astype(str).str.contains('2024')]

In [ ]:
colunas_retiradas = [
    
                     #dados informativos
                     'prontuario', 
                     'n_internacao',
                     'data_saida',
                     'total_diagnostico_principal_alta',
                     'total_diagnostico_secundario_alta',
                     'lto_lto_id', 
                     'unidade_internacao', 
                     'data_nascimento', 
                     'naturalidade', 
                     'nacionalidade', 
                     'municipio_residencia', 
                     'estado_civil',
                     'transferencias_durante_a_internacao',
                     'unidades_que_passou', 
                     'especialidades_que_passou',
    
                     #coluna com valor único
                     'SpO2__anamnese_enfermagem',
    
                     #dados futuros não são utilizados como feature
                     'internacoes_365_dias_depois_alta',
                     'internacoes_7_dias_depois_alta',
                     'ultimo_albumina', 
                     'ultimo_c_reativa', 
                     'ultimo_sodio', 
                     'ultimo_ureia', 
                     'ultimo_hemoglobina',
                     'ultimo_creatinina',
                     'ultimo_leococitos_hemograma',
                     'diagnostico_principal_alta', 
                     'diagnostico_secundario_alta']

df = df.drop(columns=colunas_retiradas)

##### **Setting training and testing data**

In [ ]:
X_train = df_treino.drop(columns = [
                       'tempo_permanencia', 
])

X_test = df_teste.drop(columns = [
                       'tempo_permanencia', 
])

y_train = df_treino['tempo_permanencia']

y_test = df_teste['tempo_permanencia']

##### **Encoding the label**

In [ ]:
y_train = y_train >= 30
y_test = y_test >= 30

##### **Separating encoding types**

In [ ]:
xtrain_onehotencoding = X_train[['FC__anamnese_medica', 
                         'FR__anamnese_medica', 
                         'PA__anamnese_medica',
                         'T__anamnese_medica', 
                         'SpO2__anamnese_medica', 
                         'R__anamnese_medica',
                         'FC__anamnese_enfermagem', 
                         'FR__anamnese_enfermagem',
                         'PA__anamnese_enfermagem', 
                         'T__anamnese_enfermagem',
                         'R__anamnese_enfermagem',
                         'SpO2__anamnese_enfermagem',
                         'raca_cor'
]]

xtest_onehotencoding = X_test[['FC__anamnese_medica', 
                        'FR__anamnese_medica', 
                         'PA__anamnese_medica',
                         'T__anamnese_medica', 
                         'SpO2__anamnese_medica', 
                         'R__anamnese_medica',
                         'FC__anamnese_enfermagem', 
                         'FR__anamnese_enfermagem',
                         'PA__anamnese_enfermagem', 
                         'T__anamnese_enfermagem',
                         'R__anamnese_enfermagem',
                         'SpO2__anamnese_enfermagem',
                         'raca_cor'
]]

xtrain_ordinalencoding = X_train['grau_instrucao']
xtest_ordinalencoding = X_test['grau_instrucao']

xtrain_fasttext = X_train[[
               'especialidade_internacao',
               'cid_primario_internacao', 
               'cid_secundario_internacao',
]]

xtest_fasttext = X_test[[
               'especialidade_internacao',
               'cid_primario_internacao', 
               'cid_secundario_internacao',
]]

xtrain_numeric = X_train[['primeiro_albumina', 
            'primeiro_c_reativa', 
            'primeiro_sodio',
            'primeiro_ureia', 
            'primeiro_hemoglobina', 
            'primeiro_creatinina',
            'primeiro_leococitos_hemograma', 
            'tempo_ultima_internacao_internacao_atual',
            'idade_internacao']]

xtest_numeric = X_test[['primeiro_albumina', 
            'primeiro_c_reativa', 
            'primeiro_sodio',
            'primeiro_ureia', 
            'primeiro_hemoglobina', 
            'primeiro_creatinina',
            'primeiro_leococitos_hemograma', 
            'tempo_ultima_internacao_internacao_atual',
            'idade_internacao']]

xtrain_sex = X_train['sexo']
xtest_sex = X_test['sexo']

In [ ]:
xtrain_ordinalencoding = pd.DataFrame(xtrain_ordinalencoding)
xtest_ordinalencoding = pd.DataFrame(xtest_ordinalencoding)

xtrain_sex = pd.DataFrame(xtrain_sex)
xtest_sex = pd.DataFrame(xtest_sex)

##### Sexo como variável binária

In [ ]:
xtrain_sex = (xtrain_sex['sexo'] == 'Masculino').astype(int)
xtest_sex = (xtest_sex['sexo'] == 'Masculino').astype(int)

OneHotEncoder

In [ ]:
ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore').set_output(transform='pandas')

ohe.fit(xtrain_onehotencoding)

os.makedirs("pickles", exist_ok=True)
with open('pickles/permanencia_alldata_onehotencoding.pkl', 'wb') as f:
    pickle.dump(ohe, f)

ohetransform_train = ohe.transform(xtrain_onehotencoding)
ohetransform_test = ohe.transform(xtest_onehotencoding)

OrdinalEncoder

In [ ]:
ore = OrdinalEncoder().set_output(transform='pandas')

ore.fit(xtrain_ordinalencoding)

os.makedirs("pickles", exist_ok=True)
with open('pickles/permanencia_alldata_ordinalencoding.pkl', 'wb') as f:
    pickle.dump(ore, f)

oretransform_train = ore.transform(xtrain_ordinalencoding) 
oretransform_test = ore.transform(xtest_ordinalencoding)

##### **FastText (CIDs e procedimentos)**

In [ ]:
guide_model = fasttext.load_model('cc.pt.300.bin')

In [ ]:
def fasttext_encoder(df, model):
    
    def vectorize_cell(cell):                
        return guide_model.get_word_vector(str(cell))

    vectorized_df = df.applymap(vectorize_cell)
    
    return vectorized_df

In [ ]:
df_encoder_xtrain = fasttext_encoder(xtrain_fasttext, guide_model)
df_encoder_fasttext_xtrain = pd.DataFrame(df_encoder_xtrain)

In [ ]:
df_encoder_xtest = fasttext_encoder(xtest_fasttext, guide_model)
df_encoder_fasttext_xtest = pd.DataFrame(df_encoder_xtest)

In [ ]:
def word2vec(line):
    words = str(line).split()
    vector = [guide_model.get_word_vector(word) for word in words]
    
    if vector:
        return np.mean(vector, axis=0)
    else:
        return np.zeros(guide_model.get_dimension) 

In [ ]:
def vectorize_columns(df, columns, word2vec):
    
    vectors = []
    vector_size = guide_model.get_dimension()

    
    for col in columns:
        
        vecs = df[col].apply(lambda x: word2vec(x)).values
        
        vecs = pd.DataFrame(np.stack(vecs))
        vecs = vecs.rename(columns={i: f'{col}_vec_{i}' for i in vecs.columns})
        
        vectors.append(vecs)
    
    return pd.concat(vectors, axis=1)

In [ ]:
columns = ['especialidade_internacao',	'cid_primario_internacao', 'cid_secundario_internacao']

xtrain_vecs = vectorize_columns(df_encoder_fasttext_xtrain, columns, word2vec)

xtest_vecs = vectorize_columns(df_encoder_fasttext_xtest, columns, word2vec)

##### **Final treating to data: Imputing null data**

In [ ]:
def string_to_nan(df):

    if isinstance(df, str):
        return np.nan

    else:
        return df

xtrain_numeric = xtrain_numeric.applymap(string_to_nan)
xtest_numeric = xtest_numeric.applymap(string_to_nan)

##### KNNImputer

In [ ]:
imputer = KNNImputer().set_output(transform='pandas')

imputer.fit(xtrain_numeric)

os.makedirs("pickles", exist_ok=True)
with open('pickles/permanencia_alldata_knnimputer.pkl', 'wb') as f:
    pickle.dump(imputer, f)

xtrain_numeric = imputer.transform(xtrain_numeric)
xtest_numeric = imputer.transform(xtest_numeric)

##### Concatenated DataFrame

1. X_train

In [ ]:
X_train = pd.concat([ohetransform_train.reset_index().drop(['index'], axis=1),
                     oretransform_train.reset_index().drop(['index'], axis=1),
                     xtrain_vecs.reset_index().drop(['index'], axis=1),
                     xtrain_numeric.reset_index().drop(['index'], axis=1),
                     xtrain_sex.reset_index().drop(['index'], axis=1),
                    ], axis=1)

2. X_test

In [ ]:
X_test = pd.concat([ohetransform_test.reset_index().drop(['index'], axis=1),
                     oretransform_test.reset_index().drop(['index'], axis=1),
                     xtest_vecs.reset_index().drop(['index'], axis=1),
                     xtest_numeric.reset_index().drop(['index'], axis=1),
                     xtest_sex.reset_index().drop(['index'], axis=1),
                    ], axis=1)

#### Imbalance Treatment: SMOTETomek and StandardScaler

1. Standard Scaling

In [ ]:
scaler = StandardScaler()
scaler.fit(X_train)

os.makedirs("pickles", exist_ok=True)
with open('pickles/permanencia_alldata_standardscaling.pkl', 'wb') as f:
    pickle.dump(scaler, f)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

2. SMOTETomek (SMOTE and then Tomek)

In [ ]:
smt = SMOTETomek(random_state=1)

X_train_resampled, y_train_resampled = smt.fit_resample(X_train_scaled, y_train)

In [ ]:
px.pie(names=y_train, color_discrete_sequence=["skyblue","black"])

In [ ]:
y_train_resampled = np.array(y_train_resampled)
y_train = np.array(y_train)

##### **Saving and exporting data**

In [ ]:
X_train_resampled = pd.DataFrame(X_train_resampled, columns=X_train.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns)

In [ ]:
X_train_resampled.to_csv("../../PREPROCESSED_FILES/tempo_perm/X_train_30TEMPOPER.csv", index=False)
X_test_scaled.to_csv("../../PREPROCESSED_FILES/tempo_perm/X_test_30TEMPOPER.csv", index=False)

np.save("../../PREPROCESSED_FILES/tempo_perm/y_train_30TEMPOPER.npy", np.array(y_train_resampled))
np.save("../../PREPROCESSED_FILES/tempo_perm/y_test_30TEMPOPER.npy", np.array(y_test))

### Preprocessing - Tempo de Permanência: Boruta data

In [ ]:
rf = RandomForestClassifier(random_state=42)

boruta = BorutaPy(rf, max_iter=1000, n_estimators='auto', verbose=2, random_state=42).fit(X_train_resampled, y_train_resampled)

In [ ]:
with open("pickles/permanencia_boruta.pkl", "wb") as f:
    pickle.dump(boruta, f)

In [ ]:
cols_selected = boruta.support_
cols_selected = X_train.columns[cols_selected]

In [ ]:
with open("pickles/permanencia_cols_selected_boruta.pkl", "wb") as f:
    pickle.dump(cols_selected, f)

In [ ]:
X_train_resampled = X_train_resampled[cols_selected]
X_test_scaled = X_test_scaled[cols_selected]

In [ ]:
X_train_resampled = pd.DataFrame(X_train_resampled)
X_test_scaled = pd.DataFrame(X_test_scaled)

##### **Saving and exporting data**

In [ ]:
X_train_resampled.to_csv("../../PREPROCESSED_FILES/tempo_perm/X_train_30TEMPOPER_Boruta.csv", index=False)
X_test_scaled.to_csv("../../PREPROCESSED_FILES/tempo_perm/X_test_30TEMPOPER_Boruta.csv", index=False)

np.save("../../PREPROCESSED_FILES/tempo_perm/y_train_30TEMPOPER_Boruta.npy", np.array(y_train_resampled))
np.save("../../PREPROCESSED_FILES/tempo_perm/y_test_30TEMPOPER_Boruta.npy", np.array(y_test))

### Preprocessing - Tempo de Permanência: Literature data

In [ ]:
df = pd.read_excel('../../../data/coorte-rues-2022-2023-V3_com_extracoes.xlsx')
df = df.drop('Unnamed: 0', axis=1)

In [ ]:
df.fillna({
    'FC__anamnese_medica': 'NORMOCARDICO',
    'FR__anamnese_medica': 'EUPNEIA',
    'PA__anamnese_medica': 'NORMOTENSAO',
    'T__anamnese_medica': 'NORMOTERMIA',
    'SpO2__anamnese_medica': 'NORMOXEMIA',
    'SpO2__anamnese_enfermagem': 'NORMOXEMIA',
    'R__anamnese_medica': 'LUCIDO E ORIENTADO',
    'FC__anamnese_enfermagem': 'NORMOCARDICO',
    'FR__anamnese_enfermagem': 'EUPNEIA',
    'PA__anamnese_enfermagem': 'NORMOTENSAO',
    'T__anamnese_enfermagem': 'NORMOTERMIA',
    'R__anamnese_enfermagem': 'LUCIDO E ORIENTADO'
}, inplace=True)

#### **Definindo uma nova variável: "cids_cardiacos"**

A variável CIDs cardíacos retorna True se o indivíduo tem algum diagnóstico cardíaco e False caso contrário. 

In [ ]:
cids_cardiacos = {
    "INSUFICIÊNCIA CARDÍACA",
    "INSUFICIÊNCIA CARDÍACA CONGESTIVA",
    "INSUFICIÊNCIA VENTRICULAR ESQUERDA",
    "INSUFICIÊNCIA CARDÍACA NÃO ESPECIFICADA"
}

def contem_cid_de_interesse(camp):
    camp = str(camp)
    
    terms = [term.strip().upper() for term in camp.split(',')]
    return any(term in cids_cardiacos for term in terms)

df['insuficiencia_cardiaca'] = df.apply(
    lambda row: contem_cid_de_interesse(row.get('cid_primario_internacao', '')) or
                contem_cid_de_interesse(row.get('cid_secundario_internacao', '')), axis=1)

#### **Selecting variables from literature**

Idade, creatinina, temperatura, ureia, leucócitos, sexo, albumina, sódio, diagnóstico, pressão sanguínea, tipo de admissão, departamento de registro, número de hospitalizações, tempo desde a última admissão, batimentos cardíacos, frequência respiratória, hemoglobina, falência cardíaca/falência cardíaca congestiva


#### **Variáveis selecionadas pela literatura, mas não utilizadas:**
    - length of stay (dado futuro)
    - insurance type (não temos)
    - departament of discharge (dado futuro e todos RUE)

In [ ]:
df = df[['idade_internacao', # age
            
            'primeiro_creatinina', # creatinine
            
            'T__anamnese_enfermagem', # temperature
            
            'primeiro_ureia', # urea
            
            'primeiro_leococitos_hemograma', # white blood cell count 
            
            'sexo', # gender

            'primeiro_albumina', # albumin
            
            'primeiro_sodio', # sodium
            
            'cid_primario_internacao', # diagnóstico
            
            'cid_secundario_internacao', # diagnostico
            
            'PA__anamnese_enfermagem', # blood pressure
            
            'especialidade_internacao', # type of admition
            
            'unidade_internacao', # department of registration
                                  
            'internacoes_ultimos_365_dias', # number of hospitalizations
            
            'tempo_ultima_internacao_internacao_atual', # time from last admition
            
            'FC__anamnese_enfermagem', # heart rate
            
            'SpO2__anamnese_enfermagem', # respiratory rate
            
            'primeiro_hemoglobina', # haemoglobin

            'insuficiencia_cardiaca', # heart failiure / congestive heart failiure

            'data_internacao', 'prontuario', 'tempo_permanencia'

]]

# length of stay (dado futuro)
# insurance type (não temos)
# departament of discharge (dado futuro e todos RUE)

##### **Setting training and testing data**

In [ ]:
df = df.sort_values(by=['prontuario', 'data_internacao'])

In [ ]:
df_treino = df[df['data_internacao'].astype(str).str.contains('2023|2022')]
df_teste = df[df['data_internacao'].astype(str).str.contains('2024')]

In [ ]:
X_train = df_treino.drop(columns = ['tempo_permanencia'])

X_test = df_teste.drop(columns = ['tempo_permanencia'])

y_train = df_treino['tempo_permanencia']

y_test = df_teste['tempo_permanencia']

Encoding the label

In [ ]:
y_train = y_train >= 30
y_test = y_test >= 30

#### **Separating encoding types**

In [ ]:
xtrain_onehotencoding = X_train[[
                         'FC__anamnese_enfermagem', 
                         'PA__anamnese_enfermagem', 
                         'T__anamnese_enfermagem',
                         'SpO2__anamnese_enfermagem'
]]
xtest_onehotencoding = X_test[[
                         'FC__anamnese_enfermagem', 
                         'PA__anamnese_enfermagem', 
                         'T__anamnese_enfermagem',
                         'SpO2__anamnese_enfermagem'
]]


xtrain_fasttext = X_train[[
               'especialidade_internacao',
               'cid_primario_internacao', 
               'cid_secundario_internacao',
]]
xtest_fasttext = X_test[[
               'especialidade_internacao',
               'cid_primario_internacao', 
               'cid_secundario_internacao',
]]


xtrain_numeric = X_train[['primeiro_albumina', 
            'primeiro_sodio',
            'primeiro_ureia', 
            'primeiro_hemoglobina', 
            'primeiro_creatinina',
            'primeiro_leococitos_hemograma', 
            'tempo_ultima_internacao_internacao_atual',
            'idade_internacao']]
xtest_numeric = X_test[['primeiro_albumina', 
            'primeiro_sodio',
            'primeiro_ureia', 
            'primeiro_hemoglobina', 
            'primeiro_creatinina',
            'primeiro_leococitos_hemograma', 
            'tempo_ultima_internacao_internacao_atual',
            'idade_internacao']]

xtrain_sex = X_train['sexo']
xtest_sex = X_test['sexo']

xtrain_cardiac = X_train['insuficiencia_cardiaca']
xtest_cardiac = X_test['insuficiencia_cardiaca']

In [ ]:
xtrain_sex = pd.DataFrame(xtrain_sex)
xtest_sex = pd.DataFrame(xtest_sex)

##### **Sex and heart failure as binary data**

In [ ]:
xtrain_sex = (xtrain_sex['sexo'] == 'Masculino').astype(int)
xtest_sex = (xtest_sex['sexo'] == 'Masculino').astype(int)

In [ ]:
xtrain_cardiac = xtrain_cardiac.astype(int)
xtest_cardiac = xtest_cardiac.astype(int)

#### **Transformando dados categóricos em números com OneHotEncoder e FastText**

In [ ]:
ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore').set_output(transform='pandas')
ohe.fit(xtrain_onehotencoding)

os.makedirs("pickles", exist_ok=True)
with open('pickles/permanencia_liter_onehotencoding.pkl', 'wb') as f:
    pickle.dump(scaler, f)

ohetransform_train = ohe.transform(xtrain_onehotencoding)
ohetransform_test = ohe.transform(xtest_onehotencoding)

In [ ]:
def fasttext_encoder(df, model):
    
    def vectorize_cell(cell):                
        return guide_model.get_word_vector(str(cell))

    vectorized_df = df.applymap(vectorize_cell)
    
    return vectorized_df

In [ ]:
df_encoder_xtrain = fasttext_encoder(xtrain_fasttext, guide_model)
df_encoder_fasttext_xtrain = pd.DataFrame(df_encoder_xtrain)

In [ ]:
df_encoder_xtest = fasttext_encoder(xtest_fasttext, guide_model)
df_encoder_fasttext_xtest = pd.DataFrame(df_encoder_xtest)

In [ ]:
def word2vec(line):
    words = str(line).split()
    vector = [guide_model.get_word_vector(word) for word in words]
    
    if vector:
        return np.mean(vector, axis=0)
    else:
        return np.zeros(guide_model.get_dimension) 

In [ ]:
def vectorize_columns(df, columns, word2vec):
    
    vectors = []
    vector_size = guide_model.get_dimension()

    
    for col in columns:
        
        vecs = df[col].apply(lambda x: word2vec(x)).values
        
        vecs = pd.DataFrame(np.stack(vecs))
        vecs = vecs.rename(columns={i: f'{col}_vec_{i}' for i in vecs.columns})
        
        vectors.append(vecs)
    
    return pd.concat(vectors, axis=1)

In [ ]:
columns = ['especialidade_internacao',	'cid_primario_internacao', 'cid_secundario_internacao']

xtrain_vecs = vectorize_columns(df_encoder_fasttext_xtrain, columns, word2vec)
xtest_vecs = vectorize_columns(df_encoder_fasttext_xtest, columns, word2vec)

##### **Tratamento final com dados: imputando dados ausentes**

In [ ]:
def string_to_nan(df):

    if isinstance(df, str):
        return np.nan

    else:
        return df

xtrain_numeric = xtrain_numeric.applymap(string_to_nan)
xtest_numeric = xtest_numeric.applymap(string_to_nan)

KNNImputer

In [ ]:
imputer = KNNImputer().set_output(transform='pandas')

xtrain_numeric = imputer.fit(xtrain_numeric).transform(xtrain_numeric)
xtest_numeric = imputer.transform(xtest_numeric)

#### **Concatenated Literature DataFrame**

1. X_train

In [ ]:
X_train = pd.concat([ohetransform_train.reset_index().drop(['index'], axis=1),
                     xtrain_vecs.reset_index().drop(['index'], axis=1),
                     xtrain_numeric.reset_index().drop(['index'], axis=1),
                     xtrain_sex.reset_index().drop(['index'], axis=1),
                     xtrain_cardiac.reset_index().drop(['index'], axis=1)
], axis=1)

2. X_test

In [ ]:
X_test = pd.concat([ohetransform_test.reset_index().drop(['index'], axis=1),
                     xtest_vecs.reset_index().drop(['index'], axis=1),
                     xtest_numeric.reset_index().drop(['index'], axis=1),
                     xtest_sex.reset_index().drop(['index'], axis=1),
                     xtest_cardiac.reset_index().drop(['index'], axis=1)
], axis=1)

#### **Imbalance Treatment: SMOTETomek and StandardScaler**

1. Standard Scaling

In [ ]:
scaler = StandardScaler()
scaler.fit(X_train)

os.makedirs("pickles", exist_ok=True)
with open('pickles/permanencia_liter_standardscaling.pkl', 'wb') as f:
    pickle.dump(scaler, f)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

2. SMOTETomek (SMOTE and then Tomek)

In [ ]:
smt = SMOTETomek(random_state=1)

X_train_resampled, y_train_resampled = smt.fit_resample(X_train_scaled, y_train)

In [ ]:
px.pie(names=y_train, color_discrete_sequence=["skyblue","black"])

In [ ]:
y_train_resampled = np.array(y_train_resampled)
y_train = np.array(y_train)

##### **Saving and exporting data**

In [ ]:
X_train_resampled = pd.DataFrame(X_train_resampled, columns=X_train.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns)

In [ ]:
X_train_resampled.to_csv("../../PREPROCESSED_FILES/tempo_perm/X_train_liter_30TEMPOPER.csv", index=False)
X_test_scaled.to_csv("../../PREPROCESSED_FILES/tempo_perm/X_test_liter_30TEMPOPER.csv", index=False)

np.save("../../PREPROCESSED_FILES/tempo_perm/y_train_liter_30TEMPOPER.npy", np.array(y_train_resampled))
np.save("../../PREPROCESSED_FILES/tempo_perm/y_test_liter_30TEMPOPER.npy", np.array(y_test))